# Base de données: interroger une base de données MySQL

## Imports

In [1]:
import os
import sqlite3
import zipfile
from pathlib import Path
import requests

# Créer un dossier data s'il n'existe pas encore
if not os.path.exists("../data"):
    os.mkdir("../data")

## Créer une base de données SQLite

In [2]:
# Créer un répertoire DB s'il n'existe pas encore
db_path = '../data/db'
if not os.path.exists(db_path):
    os.mkdir(db_path)

In [11]:
# Récupérer le fichier ZIP qui contient la DB de test
url = "https://uv.ulb.ac.be/mod/resource/view.php?id=1523394"
filename = url.split("/")[-1]
# Récupérer le fichier zip dans la RAM
response = requests.get(url)

In [12]:
# Ecrire le fichier sur le disque
with open(os.path.join(db_path, filename), 'wb') as f:
    f.write(response.content)

In [13]:
# Extraire le zip pour obtenir la db
with zipfile.ZipFile(os.path.join(db_path, filename), 'r') as zip_ref:
    zip_ref.extractall(db_path)

BadZipFile: File is not a zip file

## Schéma de la base de données

<img src="https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

## Se connecter à la base de données

In [6]:
conn = sqlite3.connect(os.path.join(db_path, "chinook.db"))
c = conn.cursor()

## Trouver l'identifiant de Miles Davis

In [7]:
artist = 'Miles Davis'

query1 = f"""
    SELECT
        artistId
    FROM
        artists
    WHERE
        name = '{artist}';
"""
c.execute(query1)
artist_id = c.fetchone()[0]

print(artist_id)

68


## Trouver les identifiants d'albums de Miles Davis

In [8]:
query2 = f"""
    SELECT
        albumId
    FROM 
        albums 
    WHERE 
        artistId = '{artist_id}';
"""
albums_ids = []
for row in c.execute(query2):
    albums_ids.append(str(row[0]))

print(albums_ids)

['48', '49', '157']


## Trouver les morceaux des albums de Miles Davis

In [9]:
query3 = f"""
    SELECT
        name,
        albumId
    FROM
        tracks
    WHERE 
        albumId IN ({",".join(albums_ids)});
"""

songs = set()
for row in c.execute(query3):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## En une seule requête...

In [10]:
query4 = f"""
    SELECT
        tracks.name,
        tracks.albumId
    FROM
        artists
        JOIN albums ON artists.artistId = albums.artistId
        JOIN tracks ON albums.albumId = tracks.albumId
    WHERE 
        artists.name = '{artist}';
"""

songs = set()
for row in c.execute(query4):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## Pour en savoir plus

- [Tutoriel SQLite](https://www.sqlitetutorial.net/)
- [The SQL cheat sheet](https://www.sqltutorial.org/wp-content/uploads/2016/04/SQL-cheat-sheet.pdf)